In [1]:
import pandas as pd
import numpy as np
import requests
import json
import os
import pprint
from citipy import citipy

In [2]:
weather_df = pd.read_csv("his_weather_final.csv", low_memory=False) #put in low_memory function due to warning
PC = pd.read_csv("Park Coordinates.csv")
pcrounded = PC.round({"Latitude": 4, "Longitude": 3})

In [3]:
def cartesian_product_basic(left, right):
    return (
       weather_df.assign(key=1).merge(pcrounded.assign(key=1), on='key').drop('key', 1))
weather_merged_df = cartesian_product_basic(weather_df, pcrounded)
#Credit: Umair Khakoo

In [4]:
weather_df_unclean = weather_merged_df[
    ((abs(weather_merged_df["Latitude_x"] - weather_merged_df["Latitude_y"])) <= 0.44) &\
    ((abs(weather_merged_df["Longitude_x"] - weather_merged_df["Longitude_y"])) <= 0.44)
]

In [5]:
weather_df_unclean

,Address,Date time,Minimum Temperature,Maximum Temperature,Temperature,Dew Point,Relative Humidity,Heat Index,Wind Speed,Wind Gust,...,Sea Level Pressure,Weather Type,Latitude_x,Longitude_x,Resolved Address,Name,Info,Latitude_y,Longitude_y,Park
56,"alamogordo,New Mexico",01/01/2015,21.1,43.4,32.1,19.7,60.46,NaN,9.6,NaN,...,1016.5,"Heavy Snow, Lightning Without Thunder, Fog",32.89925,-105.96032,"Alamogordo, NM, United States",NaN,NaN,32.7872,-106.326,White Sands National Park
118,"alamogordo,New Mexico",01/02/2015,25.5,46.3,34.5,27.5,77.24,NaN,14.4,NaN,...,1012.5,"Heavy Snow, Light Snow, Mist, Light Drizzle, R...",32.89925,-105.96032,"Alamogordo, NM, United States",NaN,NaN,32.7872,-106.326,White Sands National Park
180,"alamogordo,New Mexico",01/03/2015,25.0,44.7,34.8,25.7,72.52,NaN,15.3,NaN,...,1015.5,"Light Snow, Mist, Rain",32.89925,-105.96032,"Alamogordo, NM, United States",NaN,NaN,32.7872,-106.326,White Sands National Park
242,"alamogordo,New Mexico",01/04/2015,22.6,50.4,34.8,23.8,66.93,NaN,8.7,NaN,...,1029.2,NaN,32.89925,-105.96032,"Alamogordo, NM, United States",NaN,NaN,32.7872,-106.326,White Sands National Park
304,"alamogordo,New Mexico",01/05/2015,23.3,55.4,36.6,23.1,62.59,NaN,8.9,NaN,...,1030.9,NaN,32.89925,-105.96032,"Alamogordo, NM, United States",NaN,NaN,32.7872,-106.326,White Sands National Park
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6792444,"twentynine palms,California",12/27/2019,40.3,53.0,45.2,33.6,65.14,NaN,17.8,21.9,...,1014.1,NaN,34.13561,-116.05461,"Twentynine Palms, CA, United States",NaN,NaN,33.8734,-115.901,Joshua Tree National Park
6792506,"twentynine palms,California",12/28/2019,35.1,64.1,47.8,30.7,55.21,NaN,14.6,NaN,...,1019.0,NaN,34.13561,-116.05461,"Twentynine Palms, CA, United States",NaN,NaN,33.8734,-115.901,Joshua Tree National Park
6792568,"twentynine palms,California",12/29/2019,38.4,57.4,48.5,33.5,58.79,NaN,6.3,NaN,...,1022.3,NaN,34.13561,-116.05461,"Twentynine Palms, CA, United States",NaN,NaN,33.8734,-115.901,Joshua Tree National Park
6792630,"twentynine palms,California",12/30/2019,44.7,50.9,47.6,32.6,57.04,NaN,11.1,24.2,...,1018.5,NaN,34.13561,-116.05461,"Twentynine Palms, CA, United States",NaN,NaN,33.8734,-115.901,Joshua Tree National Park


In [6]:
weather_wanted_unclean = weather_df_unclean.drop(columns = {"Address", "Minimum Temperature",\
                                                            "Maximum Temperature", "Temperature",\
                                                            "Dew Point", "Relative Humidity", \
                                                            "Heat Index", "Wind Speed", "Wind Gust", \
                                                            "Sea Level Pressure", "Weather Type", "Name", \
                                                            "Info", "Wind Direction", "Wind Chill", "Precipitation", \
                                                            "Precipitation Cover", "Resolved Address", \
                                                            "Snow Depth", "Visibility"})
weather_dates = pd.to_datetime(weather_wanted_unclean["Date time"])
weather_date_df_bad = pd.DataFrame(weather_dates)
weather_date_df = weather_date_df_bad.rename(columns = {"Date time": "Year"})
weather_date_drop = weather_wanted_unclean.drop(columns= "Date time")
weather_df_clean = weather_date_drop.merge(weather_date_df, left_index = True, right_index = True)

In [7]:
weather_df_clean

,Cloud Cover,Latitude_x,Longitude_x,Latitude_y,Longitude_y,Park,Year
56,73.2,32.89925,-105.96032,32.7872,-106.326,White Sands National Park,2015-01-01
118,54.9,32.89925,-105.96032,32.7872,-106.326,White Sands National Park,2015-01-02
180,17.7,32.89925,-105.96032,32.7872,-106.326,White Sands National Park,2015-01-03
242,0.0,32.89925,-105.96032,32.7872,-106.326,White Sands National Park,2015-01-04
304,0.0,32.89925,-105.96032,32.7872,-106.326,White Sands National Park,2015-01-05
...,...,...,...,...,...,...,...
6792444,31.8,34.13561,-116.05461,33.8734,-115.901,Joshua Tree National Park,2019-12-27
6792506,0.0,34.13561,-116.05461,33.8734,-115.901,Joshua Tree National Park,2019-12-28
6792568,1.0,34.13561,-116.05461,33.8734,-115.901,Joshua Tree National Park,2019-12-29
6792630,55.2,34.13561,-116.05461,33.8734,-115.901,Joshua Tree National Park,2019-12-30


In [8]:
cloud_pivot = pd.pivot_table(weather_df_clean, values = "Cloud Cover", index = ["Year"], columns = ["Park"])

In [9]:
cloud_pivot_grouped = cloud_pivot.groupby(by=[cloud_pivot.index.year]).mean()
cloud_pivot_grouped

Park,Acadia National Park,Arches National Park,Big Bend National Park,Biscayne National Park,Black Canyon of the Gunnison National Park,Canyonlands National Park,Carlsbad Caverns National Park,Channel Islands National Park,Congaree National Park,Cuyahoga Valley National Park,...,Pinnacles National Park,Redwood National and State Parks,Rocky Mountain National Park,Saguaro National Park,Sequoia & Kings Canyon National Parks,Shenandoah National Park,Virgin Islands National Park,White Sands National Park,Yellowstone National Park,Zion National Park
Year,,,,,,,,,,,,,,,,,,,,,
2015,44.377808,21.405234,3.235890,25.233014,27.725205,21.405234,20.647260,12.661370,39.947123,56.375890,...,31.810411,50.064932,29.344658,3.609315,13.190411,38.087397,42.333425,18.165479,1.915068,18.388356
2016,41.175683,20.659016,2.628962,34.624317,25.918579,20.659016,16.522678,14.724590,34.269399,51.620765,...,39.933880,52.581421,23.687158,2.640164,14.538525,34.747814,40.092623,13.729508,2.582514,16.325137
2017,43.106575,16.102740,1.942192,33.796712,21.785479,16.102740,15.138219,15.423014,34.056438,58.283836,...,34.207397,53.453425,25.655890,2.307671,10.670411,37.294521,38.053203,11.259178,3.647397,15.120411
2018,29.928767,16.010440,2.641096,34.451233,22.637260,16.010440,17.149315,13.896712,38.621370,38.485479,...,33.158630,44.200548,23.257808,0.795068,16.134247,45.275890,23.885479,13.801096,5.399178,15.379589
2019,4.335890,17.761918,4.364658,34.643014,23.481918,17.761918,14.798904,18.207397,33.117534,30.424932,...,35.688219,40.982466,25.093973,4.846301,16.218904,35.080822,1.990137,12.328219,8.500274,18.994658


In [10]:
cloud_pivot_grouped.to_csv("Cloud_Cover_Data_Final.csv")